This kernel shows how to check your prediction with map and train data.

Have Fun !!

I used some nice functions and data from public kernels, thanks @kenmatsu4 and @hiro5299834
* https://www.kaggle.com/kenmatsu4/feature-store-for-indoor-location-navigation
* https://www.kaggle.com/hiro5299834/indoor-navigation-and-location-wifi-features

updated my sample submission to LB4.441 before post processing ;)

In [ ]:
import json
import re
import gc
import pickle
import itertools
import pandas as pd
import numpy as np
from glob import glob
from datetime import datetime as dt
from pathlib import Path
from tqdm import tqdm
import datetime
ts_conv = np.vectorize(datetime.datetime.fromtimestamp) # ut(10 digit) -> date

# pandas settings -----------------------------------------
pd.set_option("display.max_colwidth", 100)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.options.display.float_format = '{:,.5f}'.format

# Graph drawing -------------------------------------------
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc
from matplotlib_venn import venn2, venn2_circles
from matplotlib import animation as ani
from IPython.display import Image
from pylab import imread

plt.rcParams["patch.force_edgecolor"] = True
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

%matplotlib inline
%config InlineBackend.figure_format='retina'

# ML -------------------------------------------
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.simplefilter('ignore')

In [ ]:
# set your submission file here
# This is my private submission.
pred_df = pd.read_csv('../input/indoor-lb-6503/lb4441-raw.csv') 

In [ ]:
pred_df['site'] = [s1 for s1, s2, s3 in pred_df.site_path_timestamp.str.split('_')]
pred_df['path'] = [s2 for s1, s2, s3 in pred_df.site_path_timestamp.str.split('_')]
pred_df['timestamp'] = [s3 for s1, s2, s3 in pred_df.site_path_timestamp.str.split('_')]

In [ ]:
#https://www.kaggle.com/kenmatsu4/feature-store-for-indoor-location-navigation

class SiteInfo():
    def __init__(self, site_id, floor, input_path="../input/indoor-location-navigation/"):
        self.site_id = site_id
        self.floor = floor
        self.input_path = input_path
        assert Path(input_path).exists(), f"input_path do not exist: {input_path}"
        
    def get_site_info(self, keep_raw=False):
        floor_info_path = f"{self.input_path}/metadata/{self.site_id}/{self.floor}/floor_info.json"
        with open(floor_info_path, "r") as f:
            self.floor_info = json.loads(f.read())
            self.site_height = self.floor_info["map_info"]["height"]
            self.site_width = self.floor_info["map_info"]["width"]
            if not keep_raw:
                del self.floor_info
            
        geojson_map_path = f"{self.input_path}/metadata/{self.site_id}/{self.floor}/geojson_map.json"
        with open(geojson_map_path, "r") as f:
            self.geojson_map = json.loads(f.read())
            self.map_type = self.geojson_map["type"]
            self.features = self.geojson_map["features"]
            
            self.floor_coordinates = self.features[0]["geometry"]["coordinates"]
            self.store_coordinates = [self.features[i]["geometry"]["coordinates"] 
                                          for i in range(1, len(self.features))]
                
            if not keep_raw:
                del self.geojson_map
    
    def show_site_image(self):
        path = f"{self.input_path}/metadata/{self.site_id}/{self.floor}/floor_image.png"
        plt.imshow(imread(path), extent=[0, self.site_width, 0, self.site_height])

    def draw_polygon(self, size=8, only_floor=False):

        fig = plt.figure()
        ax = plt.subplot(111)
            
        xmax, xmin, ymax, ymin = self._draw(self.floor_coordinates, ax, calc_minmax=True)
        if not only_floor:
            self._draw(self.store_coordinates, ax, fill=True)
        plt.legend([])
        
        xrange = xmax - xmin
        yrange = ymax - ymin
        ratio = yrange / xrange
        
        self.x_size = size
        self.y_size = size*ratio

        fig.set_figwidth(size)
        fig.set_figheight(size*ratio)
        # plt.show()
        return ax
        
    def _draw(self, coordinates, ax, fill=False, calc_minmax=False):
        xmax, ymax = -np.inf, -np.inf
        xmin, ymin = np.inf, np.inf
        for i in range(len(coordinates)):
            ndim = np.ndim(coordinates[i])
            if ndim==2:
                corrd_df = pd.DataFrame(coordinates[i])
                if fill:
                    ax.fill(corrd_df[0], corrd_df[1], alpha=0.7)
                else:
                    corrd_df.plot.line(x=0, y=1, style="-", ax=ax)
                        
                if calc_minmax:
                    xmax = max(xmax, corrd_df[0].max())
                    xmin = min(xmin, corrd_df[0].min())

                    ymax = max(ymax, corrd_df[1].max())
                    ymin = min(ymin, corrd_df[1].min())
            elif ndim==3:
                for j in range(len(coordinates[i])):
                    corrd_df = pd.DataFrame(coordinates[i][j])
                    if fill:
                        ax.fill(corrd_df[0], corrd_df[1], alpha=0.6)
                    else:
                        corrd_df.plot.line(x=0, y=1, style="-", ax=ax)
                        
                    if calc_minmax:
                        xmax = max(xmax, corrd_df[0].max())
                        xmin = min(xmin, corrd_df[0].min())

                        ymax = max(ymax, corrd_df[1].max())
                        ymin = min(ymin, corrd_df[1].min())
            else:
                assert False, f"ndim of coordinates should be 2 or 3: {ndim}"
        if calc_minmax:
            return xmax, xmin, ymax, ymin
        else:
            return None
        

In [ ]:
floor_convert = {'1F' :  0, '2F' : 1, '3F' : 2, '4F' : 3, '5F' : 4, 
                     '6F' : 5, '7F' : 6, '8F' : 7, '9F' : 8,
                     'B'  : -1, 'B1' : -1, 'B2' : -2, 'B3' : -3, 
                     'BF' : -1, 'BM' : -1, 
                     'F1' : 0, 'F2' : 1, 'F3' : 2, 'F4' : 3, 'F5' : 4, 
                     'F6' : 5, 'F7' : 6, 'F8' : 7, 'F9' : 8, 'F10': 9,
                     'L1' : 0, 'L2' : 1, 'L3' : 2, 'L4' : 3, 'L5' : 4, 
                     'L6' : 5, 'L7' : 6, 'L8' : 7, 'L9' : 8, 'L10': 9, 
                     'L11': 10,
                     'G'  : 0, 'LG1': 0, 'LG2': 1, 'LM' : 0, 'M'  : 0, 
                     'P1' : 0, 'P2' : 1,}

In [ ]:
reconv = {}

for i in range(20):
    reconv[i - 5] = []
    
for k, v in floor_convert.items(): 
    reconv[v].append(k) 

In [ ]:
#https://www.kaggle.com/hiro5299834/indoor-navigation-and-location-wifi-features
train_dir='../input/indoor-navigation-and-location-wifi-features/' # v3

In [ ]:
for i, site_id in enumerate(pred_df.site.unique()):

    sample_df = pred_df.query("site == @site_id")
    train_df = pd.read_csv(f'{train_dir}/{site_id}_train.csv')
    
    for floor_id in sample_df['floor'].unique():
        floor_df = sample_df[sample_df['floor'] == floor_id].reset_index(drop=True)

        for fstr in reconv[floor_id]:
            sample_map = SiteInfo(site_id, fstr)
            try:
                sample_map.get_site_info()
            except:
                pass
            else:
                break

        plt.figure(figsize=(8, 8*sample_map.site_height / sample_map.site_width))
        #plt.subplot(121)
        
        sample_map.show_site_image()

        for _, g in floor_df.groupby("path"):
            plt.plot(g.x, g.y, "-o", alpha=0.6,  zorder=100, markersize=5 )
            
#        plt.title(f'pred {site_id}:{floor_id}')
        
        sample_train = train_df.query('f == @floor_id')
        
#        plt.subplot(122)
#        sample_map.show_site_image()
        
        #for _, g in sample_train.groupby("path"):
        #    plt.plot(g.x, g.y, "-o", alpha=0.6,  zorder=100, markersize=5)
        
        plt.scatter(sample_train.x, sample_train.y, marker='+', s=3)
        plt.title(f'train {site_id}:{floor_id}')
        
        # show all floor in this site
        break 
    #break